# NLP: fake and true news discrimination with BERT

Data collected from [Kaggle](https://www.kaggle.com/datasets/clmentbisaillon/fake-and-real-news-dataset?select=True.csv) refers to fake an true news datasets.

In [1]:
import pandas as pd

from sklearn.model_selection import train_test_split

In [3]:
fake = pd.read_csv('Fake.csv')
true = pd.read_csv('True.csv')

print('Fake news dataset: \n',fake.head())
print('\nTrue news dataset: \n',true.head())

Fake news dataset: 
                                                title  \
0   Donald Trump Sends Out Embarrassing New Year’...   
1   Drunk Bragging Trump Staffer Started Russian ...   
2   Sheriff David Clarke Becomes An Internet Joke...   
3   Trump Is So Obsessed He Even Has Obama’s Name...   
4   Pope Francis Just Called Out Donald Trump Dur...   

                                                text subject  \
0  Donald Trump just couldn t wish all Americans ...    News   
1  House Intelligence Committee Chairman Devin Nu...    News   
2  On Friday, it was revealed that former Milwauk...    News   
3  On Christmas day, Donald Trump announced that ...    News   
4  Pope Francis used his annual Christmas Day mes...    News   

                date  
0  December 31, 2017  
1  December 31, 2017  
2  December 30, 2017  
3  December 29, 2017  
4  December 25, 2017  

True news dataset: 
                                                title  \
0  As U.S. budget fight looms, Republican

In [4]:
#build dataset with training and test split

#set the text of the article as samples
fake['text'] = fake['title'] + ' ' + fake['text']
true['text'] = true['title'] + ' ' + true['text']

#set the labels
fake['label'] = 0
true['label'] = 1

#concatenate fake and true
df = pd.concat([fake, true], ignore_index=True)

#reset rows
df = df.sample(frac=1, random_state=42).reset_index(drop=True)


#training and test dataset
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['label'])

### Finetuning of BERT

In [5]:
!pip install transformers datasets torch accelerate

In [6]:
from datasets import Dataset
from transformers import AutoTokenizer

#tokenizer from BERT
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

#preprocess function for dataset
def preprocess_function(examples):
    return tokenizer(
        examples['text'],
        truncation=True,
        padding='max_length',
        max_length=128
    )

#convert dataframe in Dataset Hugging Face
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

#map on training and test sets
train_dataset = train_dataset.map(preprocess_function, batched=True)
test_dataset = test_dataset.map(preprocess_function, batched=True)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Map:   0%|          | 0/35918 [00:00<?, ? examples/s]

Map:   0%|          | 0/8980 [00:00<?, ? examples/s]

In [7]:
from transformers import AutoModelForSequenceClassification

#load model
model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [23]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

#define metrics
def compute_metrics(pred_labels, true_labels):
    precision, recall, f1, _ = precision_recall_fscore_support(true_labels, pred_labels, average='binary')
    accuracy = accuracy_score(true_labels, pred_labels)
    return {'accuracy': accuracy, 'f1': f1, 'precision': precision, 'recall': recall}


In [9]:
from transformers import TrainingArguments, Trainer

#prepare training arguments
training_args = TrainingArguments(
    output_dir='./bert-fake-news',
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=50,
    save_total_limit=1,
    report_to='none'
)

In [10]:
#fit the model
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()

/tmp/ipython-input-4011985454.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
50,0.284100
100,0.013500
150,0.001200
200,0.001600
250,0.018100
300,0.000400
350,0.021300
400,0.000400
450,0.002600
500,0.000400


TrainOutput(global_step=13470, training_loss=0.0034228730916446557, metrics={'train_runtime': 780.0587, 'train_samples_per_second': 138.136, 'train_steps_per_second': 17.268, 'total_flos': 7087817164815360.0, 'train_loss': 0.0034228730916446557, 'epoch': 3.0})

In [12]:
#predictions on test set
predictions = trainer.predict(test_dataset)
pred_labels = predictions.predictions.argmax(-1)

In [24]:
#compute metrics
compute_metrics(pred_labels, test_df['label'])

{'accuracy': 0.9998886414253898,
 'f1': 0.999883273024396,
 'precision': 1.0,
 'recall': 0.9997665732959851}

All the evaluation metrics—accuracy, precision, recall, and F1 score—are all very close to 1.0. This indicates that the model performs exceptionally well in identifying and classifying fake news. Such high values suggest that the model has learned to distinguish between true and fake content with remarkable reliability, making it highly effective for practical applications in detecting misinformation.